<a href="https://colab.research.google.com/github/scaomath/wustl-math450/blob/main/Intro_to_PyTorch_for_Numpy_users.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to PyTorch for grad students of math

## Goal

- Use `torch` to help our research.
- Train models with less than 20m params on cloud, for free.
- Train models with 50m params on cloud with a caveat.

## Components of torch

- Tensor operations.
- Neural net modules (MLP, CNN, RNN, Transformers).
- Data utils.
- Optimizers.
- fft, torchvision, torch geometric, etc.


## Reference:
- https://github.com/scaomath/wustl-math450/tree/main/Lectures
- https://github.com/phlippe/uvadlc_notebooks/blob/master/docs/tutorial_notebooks/tutorial2/Introduction_to_PyTorch_empty.ipynb

## The Basics of PyTorch

We will start with reviewing the very basic concepts of PyTorch. As a prerequisite, we recommend to be familiar with the `numpy` package as most machine learning frameworks are based on very similar concepts. If you are not familiar with numpy yet, don't worry: here is a [tutorial](https://numpy.org/devdocs/user/quickstart.html) to go through. 

So, let's start with importing PyTorch. The package is called `torch`, based on its original framework [Torch](http://torch.ch/). As a first step, we can check its version:

In [ ]:
import torch
print("Using torch", torch.__version__)

Using torch 1.8.1+cu101


At the time of writing this tutorial (mid of October 2020), the current stable version is 1.8 with GPU support (`cu` in the end). 

As in every machine learning framework, PyTorch provides functions that are stochastic like generating random numbers. However, a very good practice is to setup your code to be reproducible with the exact same random numbers. This is why we set a seed below. 

In [ ]:
torch.manual_seed(42) # Setting the seed

### Tensors

Tensors are the PyTorch equivalent to Numpy arrays, with the addition to also have support for GPU acceleration on various operation.
The name "tensor" is a generalization of concepts you already know. For instance, a vector is a 1-D tensor, and a matrix a 2-D tensor. When working with neural networks, we will use tensors of various shapes and number of dimensions.

Most common functions you know from numpy can be used on tensors as well. Actually, since numpy arrays are so similar to tensors, we can convert most tensors to numpy arrays (and back) but we don't need it too often.

#### Initialization

Let's first start by looking at different ways of creating a tensor. There are many possible options, the most simple one is to call `torch.Tensor` passing the desired shape as input argument:

In [ ]:
x = torch.randn(2, 3, 4) # or x = torch.Tensor(2, 3, 4)
print(x)

tensor([[[ 1.9269,  1.4873,  0.9007, -2.1055],
         [ 0.6784, -1.2345, -0.0431, -1.6047],
         [ 0.3559, -0.6866, -0.4934,  0.2415]],

        [[-1.1109,  0.0915, -2.3169, -0.2168],
         [-0.3097, -0.3957,  0.8034, -0.6216],
         [-0.5920, -0.0631, -0.8286,  0.3309]]])


The function `torch.Tensor` allocates memory for the desired tensor, but reuses any values that have already been in the memory. To directly assign values to the tensor during initialization, there are many alternatives including:

* `torch.zeros`: Creates a tensor filled with zeros
* `torch.ones`: Creates a tensor filled with ones
* `torch.rand`: Creates a tensor with random values uniformly sampled between 0 and 1
* `torch.randn`: Creates a tensor with random values sampled from a normal distribution with mean 0 and variance 1
* `torch.arange`: Creates a tensor containing the values $N,N+k,N+2k,...,\min\{M, N+mk\}$ for $m=\lfloor (M-N)/k\rfloor$.
* `torch.Tensor` (input list): Creates a tensor from the list elements you provide

In [ ]:
# Create a tensor from a (nested) list
x = torch.Tensor([[1, 2], [3, 4]])
print(x)

In [ ]:
shape = x.shape
print("Shape:", x.shape)

size = x.size()
print("Size:", size)

dim1, dim2, dim3 = x.size()
print("Size:", dim1, dim2, dim3)

#### Tensor to Numpy, and Numpy to Tensor

Tensors can be converted to numpy arrays, and numpy arrays back to tensors. To transform a numpy array into a tensor, we can use the function `torch.from_numpy`:

In [ ]:
np_arr = np.array([[1, 2], [3, 4]])
tensor = torch.from_numpy(np_arr)

print("Numpy array:", np_arr)
print("PyTorch tensor:", tensor)

The conversion of tensors to numpy require the tensor to be on the CPU, and not the GPU. In case you have a tensor on GPU, you need to call `.cpu()` on the tensor beforehand. Hence, you get a line like `np_arr = tensor.cpu().numpy()`.

#### Operations

Most operations that exist in numpy, also exist in PyTorch. A full list of operations can be found in the [PyTorch documentation](https://pytorch.org/docs/stable/tensors.html#), but we will review the most important ones here.

- `add` and `add_`
- `mm`
- `argmax`
- `bincount` (`accumarray` in MATLAB)
- many others.

In [ ]:
# example

In [ ]:
# bincount example
inp = torch.randint(0, 8, (5,), dtype=torch.int64)
weights = torch.linspace(0, 1, steps=5)
print(input, weights)

print(torch.bincount(inp))

print(inp.bincount(weights))

In-place operations are usually marked with a underscore postfix (e.g. "add_" instead of "add").

Another common operation aims at changing the shape of a tensor. A tensor of size (2,3) can be re-organized to any other shape with the same number of elements (e.g. a tensor of size (6), or (3,2), ...). In PyTorch, this operation is called `view`:

In [ ]:
# example of view

## "broadcastable"
The tensor involved in an operation can be automatically expanded to be of equal sizes (without making copies of the data) to let the operation go through.

General semantics Two tensors are “broadcastable”

Each tensor has at least one dimension.
When iterating over the dimension sizes, starting at the last dimension, the dimension sizes must either be equal, one of them is 1, or one of them does not exist.

Other commonly used operations include matrix multiplications, which are essential for neural networks. Quite often, we have an input vector $\mathbf{x}$, which is transformed using a learned weight matrix $\mathbf{W}$. There are multiple ways and functions to perform matrix multiplication, some of which we list below:

* `*`: element-wise product (Hardamand product).

* `torch.matmul`: Performs the matrix product over two tensors, where the specific behavior depends on the dimensions. If both inputs are matrices (2-dimensional tensors), it performs the standard matrix product. For higher dimensional inputs, the function supports broadcasting (for details see the [documentation](https://pytorch.org/docs/stable/generated/torch.matmul.html?highlight=matmul#torch.matmul)). Can also be written as `a @ b`, similar to numpy. 
* `torch.mm`: Performs the matrix product over two matrices, but doesn't support broadcasting (see [documentation](https://pytorch.org/docs/stable/generated/torch.mm.html?highlight=torch%20mm#torch.mm))
* `torch.bmm`: Performs the matrix product with a support batch dimension. If the first tensor $T$ is of shape ($b\times n\times m$), and the second tensor $R$ ($b\times m\times p$), the output $O$ is of shape ($b\times n\times p$), and has been calculated by performing $b$ matrix multiplications of the submatrices of $T$ and $R$: $O_i = T_i @ R_i$
* `torch.einsum`: Performs matrix multiplications and more (i.e. sums of products) using the Einstein summation convention. Explanation of the Einstein sum can be found in assignment 1.

## Broadcastibility



Usually, we use `torch.matmul` or `torch.bmm`. We can try a matrix multiplication with `torch.matmul` below.

In [ ]:
# examples

### Dynamic Computation Graph and Backpropagation

One of the main reasons for using PyTorch in Deep Learning projects is that we can automatically get **gradients/derivatives** of functions that we define. We will mainly use PyTorch for implementing neural networks, and they are just fancy functions. If we use weight matrices in our function that we want to learn, then those are called the **parameters** or simply the **weights**.

If our neural network would output a single scalar value, we would talk about taking the **derivative**, but you will see that quite often we will have **multiple** output variables ("values"); in that case we talk about **gradients**. It's a more general term.

Given an input $\mathbf{x}$, we define our function by **manipulating** that input, usually by matrix-multiplications with weight matrices and additions with so-called bias vectors. As we manipulate our input, we are automatically creating a **computational graph**. This graph shows how to arrive at our output from our input. 
PyTorch is a **define-by-run** framework; this means that we can just do our manipulations, and PyTorch will keep track of that graph for us. Thus, we create a dynamic computation graph along the way.

So, to recap: the only thing we have to do is to compute the **output**, and then we can ask PyTorch to automatically get the **gradients**. 

> **Note:  Why do we want gradients?** Consider that we have defined a function, a neural net, that is supposed to compute a certain output $y$ for an input vector $\mathbf{x}$. We then define an **error measure** that tells us how wrong our network is; how bad it is in predicting output $y$ from input $\mathbf{x}$. Based on this error measure, we can use the gradients to **update** the weights $\mathbf{W}$ that were responsible for the output, so that the next time we present input $\mathbf{x}$ to our network, the output will be closer to what we want.

The first thing we have to do is to specify which tensors require gradients. By default, when we create a tensor, it does not require gradients.


### Manual gradient check
https://gist.github.com/apaszke/f93a377244be9bfcb96d3547b9bc424d

## `torch.nn`

The package `torch.nn` defines a series of useful classes like linear networks layers, activation functions, loss functions etc. A full list can be found [here](https://pytorch.org/docs/stable/nn.html). In case you need a certain network layer, check the documentation of the package first before writing the layer yourself as the package likely contains the code for it already. We import it below:

The `forward` function is where the computation of the module is taken place, and is executed when you call the module (`nn = MyModule(); nn(x)`). In the init function, we usually create the parameters of the module, using `nn.Parameter`, or defining other modules that are used in the forward function. The backward calculation is done automatically, but could be overwritten as well if wanted.

Additionally to `torch.nn`, there is also `torch.nn.functional`. It contains functions that are used in network layers. This is in contrast to `torch.nn` which defines them as `nn.Modules` (more on it below), and `torch.nn` actually uses a lot of functionalities from `torch.nn.functional`. Hence, the functional package is useful in many situations, and so we import it as well here.


In [ ]:
import torch.nn as nn
import torch.nn.functional as F


## Simple classifier
We can now make use of the pre-defined modules in the `torch.nn` package, and define our own small neural network. We will use a minimal network with a input layer, one hidden layer with tanh as activation function, and a output layer. In other words, our networks should look something like this:

<center width="100%"><img src="https://sites.wustl.edu/scao/files/2021/04/small_neural_network.svg" width="300px"></center>

The input neurons are shown in blue, which represent the coordinates $x_1$ and $x_2$ of a data point. The hidden neurons including a tanh activation are shown in white, and the output neuron in red.
In PyTorch, we can define this as follows:

In [ ]:
class MyModule(nn.Module):
    
    def __init__(self):
        super(MyModule).__init__()
        # Some init for my module
        
    def forward(self, x):
        # Function for performing the calculation of the module.
        pass

# A complete pipeline

- Data preparation
- Train-Validation (Train-Test) split.
- Model.
- Choose an optimizer or write one on our own.
- Choose an scheduler or write one on our own.
- Choose the proper loss function.
- Train! (and validate at the same time).
- Inference.

In [4]:
import torch
import numpy as np

from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("dark")

import warnings
warnings.filterwarnings("ignore")

In [5]:
train = datasets.MNIST(root='./', 
                       train=True, 
                       download=True, 
                       transform = transforms.ToTensor());

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw




Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw

Processing...
Done!


In [ ]:
train_loader = DataLoader(train, batch_size=8) # take a sample from this train loader

In [ ]:
class MLP(nn.Module): # subclass of nn.Module 
    def __init__(self, 
                 input_size: int):
        '''
        __init__: initialize
        afterward, constructors
        '''
        super(MLP, self).__init__() 
        # let MLP class inherit everything from nn.Module
        self.linear0 = nn.Linear(input_size, 256)
        self.activation = nn.ReLU()
        self.linear1 = nn.Linear(256, 10)
        
    def forward(self, x): 
      # forward is a "fixed" method from nn.Module
      # this is different from @staticmethod
      # the behavior of model(x) 

        x = x.view(x.size(0), -1) # getting rid of color channel
        x1 = self.linear0(x)
        a1 = self.activation(x1)
        output = self.linear1(a1)

        return output

In [ ]:
# explicit forward
x = sample[0] # data fed into the model
print("input data: ", x.size())

x = x.view(x.size(0), -1) # getting rid of color channel
print("reshape to remove channels: ", x.size())

input_size = x.size(-1)
print("x's last dim size: ", input_size)
linear0 = nn.Linear(input_size, 256)
x1 = linear0(x)
print("after layer 1: ", x1.size())
# batch_size: dim 0 should not change in forward pass

activation = nn.ReLU()
a1 = activation(x1) # activation does not change the shape

linear1 = nn.Linear(256, 10)
output = linear1(a1)
print("output size: ", output.size())

# softmax does not need to be implemented if using
# nn.CrossEntropyLoss
softmax = nn.Softmax(dim=-1)
output_prob = softmax(output)

# Optimizer

In this class we will learn how to write an optimizer.

#### Reference: 
Final project start code: https://www.kaggle.com/scaomath/washu-math-450-sp21-final-project-starter#Final-project:-write-our-own-optimizer

In [ ]:
from torch.optim import Optimizer

In [ ]:
class SGD(Optimizer): # subclass of Optimizer
    """
    Implements the vanilla SGD simplified 
    from the torch official one for Math 450 WashU
    
    Args:
        params (iterable): iterable of parameters to optimize or dicts defining
            parameter groups
        lr (float): learning rate
        
    Example:
        >>> optimizer = SGD(model.parameters(), lr=1e-2)
        >>> optimizer.zero_grad()
        >>> loss_fn(model(input), target).backward()
        >>> optimizer.step()
    """

    def __init__(self, params, # params: model.parameters()
                       lr: float = 1e-3, # input: type = value
                       name_input: str = 'SGD'
                 ): 
        # constructor
        defaults = dict(lr=lr, name=name_input) 
        # add a default attribute that can be accessed
        super(SGD, self).__init__(params, defaults)
        self.dummy_value = 12
        self.weight_decay = 1e-5

    def step(self, closure=None): 
      # we can ignore closure for now, useful in quasi-Newton
      '''
      step(): w_{k+1} = w_k - alpha*grad f(w_k)
      '''  
      for group in self.param_groups: # fixed in template
          '''
          remark: self.param_groups is a list of len = 1
          group = self.param_groups[0]
          '''

          for param in group['params']:
              # this if block is for debugging purpose
              # sometimes we can "freeze" some weights
              # param.grad will be None
              if param.grad is None:
                  continue
              
              # if loss.backward() is performed
              # every param will have param.grad
              # .data is like retrieving the value witout triggering autograd
              # grad_param = gradient of loss with respect to this specifc weight
              grad_param = param.grad.data
              
              # .data is like "with torch.no_grad():"
              param.data = param.data - group['lr']*grad_param

      return loss

    def get_lr(self):
      # input: self, referencing the class itself
      # this function has access to every attribute of this class
      return self.defaults['lr']

In [ ]:
loss_func = nn.CrossEntropyLoss()
epochs = 2
learning_rate = 1e-3

In [ ]:
optimizer = SGD(model.parameters(), lr=learning_rate)

In [ ]:
# pipeline
for epoch in range(epochs):
    
    model.train() # formalism, useful when we have dropout
    
    loss_vals = []
    
    with tqdm(total=len(train_loader)) as pbar: # progress bar
      for data, targets in train_loader:
          
        # forward pass
        outputs = model(data)
        
        # loss function
        loss = loss_func(outputs, targets)
        
        # record loss function values .item()
        loss_vals.append(loss.item())
        
        # clean the gradient from last iteration
        # param.grad is not zero in last iteration
        optimizer.zero_grad()
        
        # backprop
        loss.backward()
        
        # stochastic gradient descent
        # no with torch.no_grad(): block, param operation is using .data
        optimizer.step()
        
        # check accuracy

        # tqdm template
        desc = f"epoch: [{epoch+1}/{epochs}] loss: {np.mean(loss_vals):.4f}"
        pbar.set_description(desc)
        pbar.update()